In [1]:
# Install libraries within the notebook scope
sc.install_pypi_package("boto3")
sc.install_pypi_package("pandas")
sc.install_pypi_package("requests")
#sc.install_pypi_package("s3fs")
sc.install_pypi_package("fsspec")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1603754855176_0005,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/59/91/c3ac686983570cbaf01063956e51ebd547d2e496f3e33052b7a9102c9a75/botocore-1.19.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/25/47/22fc373440e144e2111363adaa07abb09ec1f03fbc071b6d9fc0bbf65f68/pandas-1.1.3-cp37-cp37m-manylinux1_x86_64.whl


  Using cached https://files.pythonhosted.org/packages/a5/8b/1df260f860f17cb08698170153ef7db672c497c1840dcc8613ce26a8a005/fsspec-0.8.4-py3-none-any.whl

In [2]:
import boto3
from datetime import datetime
import fsspec
import pandas as pd
from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as f, types as t, Window
from pathlib import Path
import re
import requests
# import s3fs
import subprocess
import timeit
from urllib.parse import urlparse

# Removes truncation of columns, column values in Pandas
# by default
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

# Monkey patching the DataFrame transform method for Spark 2.4
# This is available by default in Spark 3.0
def transform(self, f):
    return f(self)
DataFrame.transform = transform

# Override the timeit template to return the command's
# return value in addition to the time
# Reference: https://stackoverflow.com/questions/24812253/how-can-i-capture-return-value-with-python-timeit-module
timeit.template = """
def inner(_it, _timer{init}):
    {setup}
    _t0 = _timer()
    for _i in _it:
        retval = {stmt}
    _t1 = _timer()
    return _t1 - _t0, retval
"""

def shell_cmd(cmd):
    """
    Wrapper for running shell commands and printing the output
    Some helpful recipes:
    - List files on hdfs: shell_cmd("hdfs dfs -ls hdfs:///tmp/data/")
    - Remove files from hdfs: shell_cmd("hdfs dfs -rm -r hdfs:///tmp/data/test_pyspark")
    """
    for line in subprocess.check_output(cmd, shell=True).split(b'\n'):
        print(line)

def timer_method(cmd):
    """
    Wrapper for timeit that returns the value of a function and its runtime
    To use, pass a string of the function you wish to time
    Example: 
     run_time, result = timer_method("myfunction(arg1, arg2)")
    """
    # Setting globals = globals() enables the timeit function
    # to return the value generated by cmd
    return timeit.timeit(cmd, number=1, globals = globals())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Set your s3 bucket name
This should be data-scale-oreilly-{your name}   
If you dont remember check the [S3 console](https://s3.console.aws.amazon.com/)

In [3]:
MY_BUCKET_NAME = "data-scale-oreilly"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Ingesting from an S3 bucket - NYC Taxi Data

https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
* Taxi data 
* Data dictionaries 
* Taxi zone lookup table

Data ingestion has the ultimate goal of collecting, aggregating, and surfacing data for a specific purpose; an analysis, an API, a dashboard, etc. Think about how you might use the taxi data to answer the following questions:

1. Which borough is the most popular pickup or drop off spot?
1. Are green taxis more popular for trips within the same borough vs yellow taxis?
1. Build a recommendation engine that predicts surge pricing for a given time of day based on historical data  

With this in mind, lets work through bringing this data onto the cluster

In [4]:
# Note, if you copy the link from the taxi data website you will see:
# https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.csv
# Two things - first, the portion of the URL following "aws.com" is the 
# bucket name. Second, in "trip+data" the "+" is a space
taxi_data_path = "s3://nyc-tlc/trip data/yellow_tripdata_2020-01.csv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Pandas uses s3fs to read_csv from s3:
pd.read_csv(taxi_data_path, keep_default_na=False)
pd_df_taxi.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Install s3fs to access S3
Traceback (most recent call last):
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/pandas/io/parsers.py", line 686, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/pandas/io/parsers.py", line 435, in _read
    filepath_or_buffer, encoding, compression
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/pandas/io/common.py", line 222, in get_filepath_or_buffer
    filepath_or_buffer, mode=mode or "rb", **(storage_options or {})
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/fsspec/core.py", line 438, in open
    **kwargs
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/fsspec/core.py", line 287, in open_files
    expand=expand,
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/fsspec/core.py", line 600, in get_fs_token_paths
    cls = get_filesystem_class(protocol)
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/fsspec/registry.py", line 204, in get_f

In [6]:
# For reference, look at the Spark DataFrameReader, csv:
# https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html
ps_df_taxi = spark.read.option('header', True).option('inferSchema', True).csv(taxi_data_path)
ps_df_taxi.show()
ps_df_taxi.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2020-01-01 00:28:15|  2020-01-01 00:33:03|              1|          1.2|         1|                 N|         238|         239|           1|        6.0|  3.0|    0.5|      1.47|         0.0|                  0.3

In [7]:
# Talk through ingest practices around retaining original data vs augmenting
# For example, we may want to keep the data in its default format so we can
# refer back to it if there are bugs in our data ingestion code
ps_df_taxi.write.option("header", True).csv("hdfs:///tmp/input/taxi_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Discuss how spark writes files out
shell_cmd("hdfs dfs -ls hdfs:///tmp/input/taxi_data")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

b'Found 17 items'
b'-rw-r--r--   1 livy hadoop          0 2020-10-26 23:31 hdfs:///tmp/input/taxi_data/_SUCCESS'
b'-rw-r--r--   1 livy hadoop   43084929 2020-10-26 23:31 hdfs:///tmp/input/taxi_data/part-00000-c0c929e9-d133-4f0a-b45b-ad1ead6bde1e-c000.csv'
b'-rw-r--r--   1 livy hadoop   43034410 2020-10-26 23:31 hdfs:///tmp/input/taxi_data/part-00001-c0c929e9-d133-4f0a-b45b-ad1ead6bde1e-c000.csv'
b'-rw-r--r--   1 livy hadoop   43025344 2020-10-26 23:31 hdfs:///tmp/input/taxi_data/part-00002-c0c929e9-d133-4f0a-b45b-ad1ead6bde1e-c000.csv'
b'-rw-r--r--   1 livy hadoop   43007863 2020-10-26 23:31 hdfs:///tmp/input/taxi_data/part-00003-c0c929e9-d133-4f0a-b45b-ad1ead6bde1e-c000.csv'
b'-rw-r--r--   1 livy hadoop   42953869 2020-10-26 23:31 hdfs:///tmp/input/taxi_data/part-00004-c0c929e9-d133-4f0a-b45b-ad1ead6bde1e-c000.csv'
b'-rw-r--r--   1 livy hadoop   42984227 2020-10-26 23:31 hdfs:///tmp/input/taxi_data/part-00005-c0c929e9-d133-4f0a-b45b-ad1ead6bde1e-c000.csv'
b'-rw-r--r--   1 livy hadoop 

In [9]:
column_subset = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'fare_amount', 'tip_amount']
ps_df_taxi.select(*column_subset).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------------+---------------+-------------+------------+------------+-----------+----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|PULocationID|DOLocationID|fare_amount|tip_amount|
+--------------------+---------------------+---------------+-------------+------------+------------+-----------+----------+
| 2020-01-01 00:28:15|  2020-01-01 00:33:03|              1|          1.2|         238|         239|        6.0|      1.47|
| 2020-01-01 00:35:39|  2020-01-01 00:43:04|              1|          1.2|         239|         238|        7.0|       1.5|
| 2020-01-01 00:47:41|  2020-01-01 00:53:52|              1|          0.6|         238|         238|        6.0|       1.0|
| 2020-01-01 00:55:23|  2020-01-01 01:00:14|              1|          0.8|         238|         151|        5.5|      1.36|
| 2020-01-01 00:01:58|  2020-01-01 00:04:16|              1|          0.0|         193|         193|        3.5|       0.0|
| 2020-0

In [10]:
ps_df_taxi.select(*column_subset).describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+------------------+-----------------+------------------+------------------+
|summary|   passenger_count|     trip_distance|      PULocationID|     DOLocationID|       fare_amount|        tip_amount|
+-------+------------------+------------------+------------------+-----------------+------------------+------------------+
|  count|           6339567|           6405008|           6405008|          6405008|           6405008|           6405008|
|   mean|1.5153326717739555|2.9296439333097024|164.73225778952968|162.6626908194338|12.694108119771682|2.1893418306424848|
| stddev|1.1515942134278125| 83.15910597325063|  65.5437394411178| 69.9126062949608|12.127295340046489|2.7600283923784095|
|    min|                 0|            -30.62|                 1|                1|           -1238.0|             -91.0|
|    max|                 9|         210240.07|               265|              265|            4265.0|            1100.0|
+-------+-------

### Lab 2.1 - Write an ingestion function that does the following:
Given a file path to a taxi data csv (i.e. s3://nyc-tlc/trip data/green_tripdata_2020-01.csv) create a function that does the following:
1. Read the file into a Spark dataframe
1. Limit to the `column_subset` columns
1. Write the data as json to hdfs in append mode to `hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json`

Function signature:  
`def ingest_taxi_data(file_name)`

See the subsequent cell for more info on how the `ingest_taxi_data` function will be used

In [11]:
def ingest_taxi_data(file_name):
    # Enclosing code in () allows multi line
    (spark
         .read
         .option('header', True)
         .option("inferSchema", True)
         .csv(file_name)
         .select(*column_subset)           
         .write
         .mode("append")
         .json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Run the ingest for several files
taxi_data_prefix = "s3://nyc-tlc/trip data"
taxi_data_files = ["yellow_tripdata_2019-01.csv", "yellow_tripdata_2018-01.csv"]
for file_name in taxi_data_files: 
    taxi_data_path = f"{taxi_data_prefix}/{file_name}"
    ingest_taxi_data(taxi_data_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# How did the types fare in this conversion?
# https://issues.apache.org/jira/browse/SPARK-26325
# https://stackoverflow.com/questions/53697388/interpret-timestamp-fields-in-spark-while-reading-json
df = spark.read.json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
df.printSchema()
df.show(5, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DOLocationID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- trip_distance: double (nullable = true)

+------------+------------+-----------+---------------+----------+------------------------+------------------------+-------------+
|DOLocationID|PULocationID|fare_amount|passenger_count|tip_amount|tpep_dropoff_datetime   |tpep_pickup_datetime    |trip_distance|
+------------+------------+-----------+---------------+----------+------------------------+------------------------+-------------+
|68          |237         |19.0       |1              |5.0       |2018-01-03T11:35:06.000Z|2018-01-03T11:06:45.000Z|3.5          |
|162         |90          |12.0       |1              |3.0       |2018-01-03T12:04:36.000Z|2018-01-03T11:4

#### Transforming data types 

Available pyspark types are listed in the pyspark.sql.types module https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.types

This is imported as t, so to apply the IntegerType use t.IntegerType()

For pandas, see the following resources on converting types https://stackoverflow.com/questions/15891038/change-column-type-in-pandas

In [14]:
# Pyspark
(df.select("tpep_dropoff_datetime")
 .withColumn("tpep_dropoff_datetime", f.col("tpep_dropoff_datetime").cast(t.TimestampType()))
).dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('tpep_dropoff_datetime', 'timestamp')]

In [15]:
# Casting pandas columns to a type - this will give an error on empty cells
# (pd_df_taxi[[*column_subset]]
#         .astype({'passenger_count': 'Int64'}))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# To convert to Integer using pandas, we have to first deal with the null values
# to_numeric with 'coerce' will fill invalid integer values with np.NaN
# the Int64 type in later versions of pandas will convert np.NaN to a nullable
# integer type: https://pandas.pydata.org/pandas-docs/stable/user_guide/integer_na.html
# pd.to_numeric(pd_df_taxi.passenger_count, errors='coerce').astype('Int64').unique()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Modified taxi_data_ingest with transformed timestamps
def ingest_taxi_data(file_name):
    # Enclosing code in () allows multi line
    (spark
         .read
         .option('header', True)
         .option("inferSchema", True)
         .csv(file_name)
         .select(*column_subset)
         .withColumn("tpep_pickup_datetime", f.col("tpep_pickup_datetime").cast(t.TimestampType()))
         .withColumn("tpep_dropoff_datetime", f.col("tpep_dropoff_datetime").cast(t.TimestampType()))
         .write
         .mode("append")
         .json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Remove previous data
shell_cmd("hdfs dfs -rm -r hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json'
b''

In [19]:
taxi_data_prefix = "s3://nyc-tlc/trip data"
taxi_data_files = ["yellow_tripdata_2019-01.csv", "yellow_tripdata_2018-01.csv"]
for file_name in taxi_data_files: 
    taxi_data_path = f"{taxi_data_prefix}/{file_name}"
    ingest_taxi_data(taxi_data_path)
    
df = spark.read.json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DOLocationID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- trip_distance: double (nullable = true)

### Testing ingestion code

The `ingest_taxi_data` method is not well structured for testing:
* Writes to the file system
* Requires an input file to test
* What other shortcomings?

To make this code more testable, split out the transformation logic so it can be unit tested.  
Definining a transformation function that takes a dataframe and returns a dataframe provides a better interface for unit testing, and a more extensible structure in case we need to add more dataframe functions before or after the transformation step.

In [20]:
def transform_taxi_data(df):
    return (df
            .withColumn("tpep_pickup_datetime", f.col("tpep_pickup_datetime").cast(t.TimestampType()))
            .withColumn("tpep_dropoff_datetime", f.col("tpep_dropoff_datetime").cast(t.TimestampType()))
           )

# Option 1
def ingest_taxi_data_method(file_name):
    df_input = (spark
         .read
         .option('header', True).csv(taxi_data_path)
         .select(*column_subset))
    
    (transform_taxi_data(df_input)
         .write
         .mode("append")
         .json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
    )

# Option 2
def ingest_taxi_data_transform(file_name):
    # Requires patching of Dataframe.transform method in Spark 2.4, but available natively
    # in Spark 3.0 https://mungingdata.com/pyspark/chaining-dataframe-transformations/
    df_input = (spark
         .read
         .option('header', True).csv(taxi_data_path)
         .select(*column_subset)
         .transform(transform_taxi_data)
         .write
         .mode("append")
         .json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
test_data = [
    "{'tpep_pickup_datetime': '2020-05-23 21:05:23', 'tpep_dropoff_datetime': '2020-05-23 08:05:23'}",
    "{'tpep_pickup_datetime': '2020-10-01 01:05:23', 'tpep_dropoff_datetime': '2020-10-01 02:05:23'}",
    "{'tpep_pickup_datetime': '2020-02-02 15:22:23', 'tpep_dropoff_datetime': '2020-02-03 15:44:23'}"
]
expected_types = {'tpep_dropoff_datetime': 'timestamp', 'tpep_pickup_datetime': 'timestamp'}

test_df = spark.read.json(sc.parallelize(test_data))
print(test_df.dtypes)
test = transform_taxi_data(test_df)
test_types = {item[0]:item[1] for item in test.dtypes}

print(expected_types == test_types)

test.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('tpep_dropoff_datetime', 'string'), ('tpep_pickup_datetime', 'string')]
True
root
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)

#### Lets try running the ingestion code on the other taxi data sets

In [22]:
# Try using the ingest code we created for yellow taxi for all the taxis
# This will fail because the datetime fields have different names across different servcies

taxi_data_prefix = "s3://nyc-tlc/trip data/"
taxi_data_files = ["green_tripdata_2020-01.csv", "fhv_tripdata_2020-01.csv", "fhvhv_tripdata_2020-01.csv"]
for file_name in taxi_data_files: 
    taxi_data_path = f"{taxi_data_prefix}{file_name}"
    ingest_taxi_data_transform(file_name)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"cannot resolve '`tpep_pickup_datetime`' given input columns: [mta_tax, trip_distance, total_amount, payment_type, ehail_fee, trip_type, fare_amount, tip_amount, extra, congestion_surcharge, passenger_count, improvement_surcharge, tolls_amount, lpep_pickup_datetime, PULocationID, store_and_fwd_flag, lpep_dropoff_datetime, DOLocationID, VendorID, RatecodeID];;\n'Project ['tpep_pickup_datetime, 'tpep_dropoff_datetime, passenger_count#1042, trip_distance#1043, PULocationID#1040, DOLocationID#1041, fare_amount#1044, tip_amount#1047]\n+- Relation[VendorID#1035,lpep_pickup_datetime#1036,lpep_dropoff_datetime#1037,store_and_fwd_flag#1038,RatecodeID#1039,PULocationID#1040,DOLocationID#1041,passenger_count#1042,trip_distance#1043,fare_amount#1044,extra#1045,mta_tax#1046,tip_amount#1047,tolls_amount#1048,ehail_fee#1049,improvement_surcharge#1050,total_amount#1051,payment_type#1052,trip_type#1053,congestion_surcharge#1054] csv\n"
Traceback (most recent call last):
  File "<stdin>", line 27, in in

### Data modeling
Create a model for all taxi data, given that there are differences across the services in the kind of data collected

Commmon fields across all services:
* PULocationID
* DOLocationID

Fields we want to normalize across all services - this data is in all services but is named differently
* pickup datetime
* drop off datetime

Service specific fields. These are only in green or yellow data
* Passenger_count
* Trip_distance
* Fare_amount
* Tip_amount


### Lab 2.2 - Write different transformation functions for each taxi service type to match the following signature and schema:
See the [Taxi data website](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page) for reference

Fields / Types:

* pickup_datetime Timestamp
* dropoff_datetime Timestamp
* passenger_count Integer
* fare_amount Float
* tip_amount Float
* trip_distance Float
* PULocationID Integer
* DOLocationID Integer

`def transform_function(dataframe):  
    return transformed_dataframe
`

Once the transformation functions are done, rewrite `ingest_taxi_data` to use these new functions depending on the file being processed

In [23]:
column_subset

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'fare_amount', 'tip_amount']

In [24]:
def transform_yellow_taxi(df):
    subset = ['pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'fare_amount', 'tip_amount']
    return (df.withColumn("pickup_datetime", f.col("tpep_pickup_datetime").cast(t.TimestampType()))
        .withColumn("dropoff_datetime", f.col("tpep_dropoff_datetime").cast(t.TimestampType()))
        .select(*subset)
        )
        
        
def transform_green_taxi(df):
    subset = ['pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'fare_amount', 'tip_amount']
    return (df.withColumn("pickup_datetime", f.col("lpep_pickup_datetime").cast(t.TimestampType()))
        .withColumn("dropoff_datetime", f.col("lpep_dropoff_datetime").cast(t.TimestampType()))
        .select(*subset)
        )

def transform_fhv(df):
    return df.select(*["pickup_datetime", "dropoff_datetime", "PULocationID", "DOLocationID"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now that we have the ability to read all the taxi data into the same dataset, how will we be able to tell where the original data came from? The file name provides information including:
* Service type (yellow, green, etc)
* File date

We want to augment the taxi data with this information so we can refer back to it in analysis.

Is there other data we might want to augment the raw data with? Some things to consider:
* Additional fields that could help with analysis
* Metadata, like when the record was last updated

In [25]:
# Using matched groups, we can extract information from the taxi file names
# Add example file name here
TAXI_DATA_PATTERN = "(?P<service>[a-zA-Z0-9]+)_tripdata_(?P<year>[0-9]{4})-(?P<month>[0-9]{2}).csv"

def extract_file_info(file_name):
    m = re.match(TAXI_DATA_PATTERN, file_name)
    if m is not None:
        return (m.group(1), m.group(2), m.group(3))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
def ingest_taxi_data_multi_service(file_name, ingested_on):
    print(f"Processing {file_name}")
    (service, year, month) = extract_file_info(Path(file_name).name)
    input_df = spark.read.option('header', True).option('inferSchema', True).csv(file_name)
    
    if service == 'yellow':
        df_transform = transform_yellow_taxi(input_df)
    elif service == 'green':
        df_transform = transform_green_taxi(input_df)
    else:
        # FHV. What happens if there are more taxi services added?
        df_transform = transform_fhv(input_df)

    (df_transform
         .withColumn("service", f.lit(service))
         .withColumn("year", f.lit(year))
         .withColumn("month", f.lit(month))
         .withColumn("ingested_on", f.lit(ingest_timestamp))
         .write
         .mode("append")
         .json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
    )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
shell_cmd("hdfs dfs -rm -r hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

b'Deleted hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json'
b''

In [28]:
ingest_timestamp = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S%z")
taxi_data_prefix = "s3://nyc-tlc/trip data/"
taxi_data_files = ["yellow_tripdata_2020-01.csv", "green_tripdata_2020-01.csv", "fhv_tripdata_2020-01.csv", "fhvhv_tripdata_2020-01.csv"]
for file_name in taxi_data_files: 
    taxi_data_path = f"{taxi_data_prefix}{file_name}"
    ingest_taxi_data_multi_service(taxi_data_path, ingest_timestamp)
    
df_taxi_output = spark.read.json("hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json")
print(df_taxi_output.dtypes)
df_taxi_output.groupby("service").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Processing s3://nyc-tlc/trip data/yellow_tripdata_2020-01.csv
Processing s3://nyc-tlc/trip data/green_tripdata_2020-01.csv
Processing s3://nyc-tlc/trip data/fhv_tripdata_2020-01.csv
Processing s3://nyc-tlc/trip data/fhvhv_tripdata_2020-01.csv
[('DOLocationID', 'bigint'), ('PULocationID', 'bigint'), ('dropoff_datetime', 'string'), ('fare_amount', 'double'), ('ingested_on', 'string'), ('month', 'string'), ('passenger_count', 'bigint'), ('pickup_datetime', 'string'), ('service', 'string'), ('tip_amount', 'double'), ('trip_distance', 'double'), ('year', 'string')]
+-------+--------+
|service|   count|
+-------+--------+
| yellow| 6405008|
|    fhv| 1958936|
|  green|  447770|
|  fhvhv|20569325|
+-------+--------+

### Handling bad data
How to design for the inevitability of bad data  
Reference: https://blog.knoldus.com/apache-spark-handle-corrupt-bad-records/

In [29]:
bad_data = [
    "{'pickup_datetime': '2020-05-23 21:05:23', 'fare_amount': '0.05'}",
    "{'pickup_datetime': '2020-05-23 08:05:23', 'fare_amount': '10.05'}",
    "{'pickup_datetime': '2020-05-23 21:05:23', 'fare_amount}"
]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
corrupt_df = spark.read.json(sc.parallelize(bad_data), mode="PERMISSIVE", columnNameOfCorruptRecord="_corrupt_record")
corrupt_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+-------------------+
|     _corrupt_record|fare_amount|    pickup_datetime|
+--------------------+-----------+-------------------+
|                null|       0.05|2020-05-23 21:05:23|
|                null|      10.05|2020-05-23 08:05:23|
|{'pickup_datetime...|       null|               null|
+--------------------+-----------+-------------------+

In [31]:
corrupt_df = spark.read.json(sc.parallelize(bad_data), mode="DROPMALFORMED", columnNameOfCorruptRecord="_corrupt_record")
corrupt_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------------+
|fare_amount|    pickup_datetime|
+-----------+-------------------+
|       0.05|2020-05-23 21:05:23|
|      10.05|2020-05-23 08:05:23|
+-----------+-------------------+

In [32]:
corrupt_df = spark.read.json(sc.parallelize(bad_data), mode="FAILFAST", columnNameOfCorruptRecord="_corrupt_record")
corrupt_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error occurred while calling o547.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 58.0 failed 4 times, most recent failure: Lost task 15.3 in stage 58.0 (TID 657, ip-172-31-5-230.ec2.internal, executor 4): org.apache.spark.SparkException: Malformed records are detected in schema inference. Parse Mode: FAILFAST.
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:66)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:53)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441)
	at scala.collection.Iterator$class.isEmpty(Iterator.scala:331)
	at scala.collection.AbstractIterator.isEmpty(Iterator.scala:1334)
	at scala.collection.TraversableOnce$class.reduceLeftOption(TraversableOnce.scala:203)
	at scala.collection.AbstractIterator.reduceLeftOption(Ite

### Lab 2.3 - Write an ingestion for the taxi zone lookup
File location - Yes, there is a space between taxi and the '_'  

s3://nyc-tlc/misc/taxi _zone_lookup.csv

`def ingest_taxi_lookup():`
1. Read taxi lookup data, ensuring data types are correct
1. Add relevant metadata
1. Save to hdfs:///tmp/data/nyc-taxi/zone-lookup/output/section2/json
1. What write mode should be used?

Refer back to Taxi Data page for more info: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [33]:
def taxi_zone_transform(df):
    return df.withColumn("ingested_on", f.lit(ingest_timestamp))

def ingest_taxi_lookup(ingest_timestamp):
    (spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("s3://nyc-tlc/misc/taxi _zone_lookup.csv")
    .transform(taxi_zone_transform)
    .write
    .mode("overwrite")
    .json("hdfs:///tmp/data/nyc-taxi/zone-lookup/output/section2/json"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
ingest_timestamp = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S%z")
print(timer_method("ingest_taxi_lookup(ingest_timestamp)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'ingest_taxi_lookup' is not defined
Traceback (most recent call last):
  File "<stdin>", line 58, in timer_method
  File "/usr/lib64/python3.7/timeit.py", line 232, in timeit
    return Timer(stmt, setup, timer, globals).timeit(number)
  File "/usr/lib64/python3.7/timeit.py", line 176, in timeit
    timing = self.inner(it, self.timer)
  File "<timeit-src>", line 6, in inner
NameError: name 'ingest_taxi_lookup' is not defined



# Break

### Lab 2.4 - Case Study 1: Month over month, get the total count of of pickups per borough

In [4]:
taxiPath = "hdfs:///tmp/data/nyc-taxi/taxi-data/output/section2/json/"
taxiLookupPath = "hdfs:///tmp/data/nyc-taxi/zone-lookup/output/section2/json/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Join boroughs
# Expected error cartesian join. most likely a carryover bug from 2.0
spark.conf.set("spark.sql.crossJoin.enabled", "false") #<-- default
df_taxi = spark.read.json(taxiPath)
df_taxi_lookup = spark.read.json(taxiLookupPath)

taxi_filtered = (df_taxi
 .filter(df_taxi.pickup_datetime.isNotNull())
 .filter(df_taxi.dropoff_datetime.isNotNull()))
taxi_pu = (taxi_filtered
.join(df_taxi_lookup
       .select("LocationID", "Borough")
       .withColumnRenamed("Borough", "PUBorough"), 
       df_taxi_lookup.LocationID == df_taxi.PULocationID))
taxi = (taxi_pu.join(df_taxi_lookup
       .select("LocationID", "Borough")
       .withColumnRenamed("Borough", "DOBorough"), 
       df_taxi_lookup.LocationID == taxi_pu.DOLocationID))
taxi_pu.show()
taxi.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Detected implicit cartesian product for INNER join between logical plans\nJoin Inner, ((LocationID#1710L = PULocationID#1680L) && (LocationID#1710L = DOLocationID#1679L))\n:- Filter ((isnotnull(pickup_datetime#1686) && isnotnull(dropoff_datetime#1681)) && ((isnotnull(PULocationID#1680L) && (DOLocationID#1679L = PULocationID#1680L)) && isnotnull(DOLocationID#1679L)))\n:  +- Relation[DOLocationID#1679L,PULocationID#1680L,dropoff_datetime#1681,fare_amount#1682,ingested_on#1683,month#1684,passenger_count#1685L,pickup_datetime#1686,service#1687,tip_amount#1688,trip_distance#1689,year#1690] json\n+- Project [LocationID#1710L, Borough#1709 AS PUBorough#1721]\n   +- Filter isnotnull(LocationID#1710L)\n      +- Relation[Borough#1709,LocationID#1710L,Zone#1711,ingested_on#1712,service_zone#1713] json\nand\nProject [LocationID#1772L, Borough#1771 AS DOBorough#1768]\n+- Relation[Borough#1771,LocationID#1772L,Zone#1773,ingested_on#1774,service_zone#1775] json\nJoin condition is missing or trivial.

In [37]:
spark.conf.set("spark.sql.crossJoin.enabled", "true")
df_taxi = spark.read.json(taxiPath)
df_taxi_lookup = spark.read.json(taxiLookupPath)

taxi_filtered = (df_taxi
 .filter(df_taxi.pickup_datetime.isNotNull())
 .filter(df_taxi.dropoff_datetime.isNotNull()))
taxi_pu = (taxi_filtered
.join(df_taxi_lookup
       .select("LocationID", "Borough")
       .withColumnRenamed("Borough", "PUBorough"), 
       df_taxi_lookup.LocationID == df_taxi.PULocationID))
taxi = (taxi_pu.join(df_taxi_lookup
       .select("LocationID", "Borough")
       .withColumnRenamed("Borough", "DOBorough"), 
       df_taxi_lookup.LocationID == taxi_pu.DOLocationID))
taxi_pu.show()
taxi.explain()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------+--------------------+-----------+-------------------+-----+---------------+--------------------+-------+----------+-------------+----+----------+---------+
|DOLocationID|PULocationID|    dropoff_datetime|fare_amount|        ingested_on|month|passenger_count|     pickup_datetime|service|tip_amount|trip_distance|year|LocationID|PUBorough|
+------------+------------+--------------------+-----------+-------------------+-----+---------------+--------------------+-------+----------+-------------+----+----------+---------+
|          90|         148|2020-01-01T01:02:...|       null|2020-10-26 23:33:52|   01|           null|2020-01-01T00:45:...|  fhvhv|      null|         null|2020|       148|Manhattan|
|          79|         114|2020-01-01T00:53:...|       null|2020-10-26 23:33:52|   01|           null|2020-01-01T00:47:...|  fhvhv|      null|         null|2020|       114|Manhattan|
|         125|           4|2020-01-01T00:21:...|       null|2020-10-26 23:33:52|   01

In [38]:
def get_monthly_totals_pyspark(taxiPath, taxiLookupPath):
    taxi = spark.read.json(taxiPath)
    taxi_lookup = spark.read.json(taxiLookupPath)
    taxi_filtered = (taxi
     .filter(taxi.pickup_datetime.isNotNull())
     .filter(taxi.dropoff_datetime.isNotNull()))
                     
    groupDF = taxi_filtered.join(taxi_lookup, taxi_filtered.PULocationID == taxi_lookup.LocationID)
    groupDF.select("ingested_on").show() # expected error
    return groupDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
print(timer_method("get_monthly_totals_pyspark(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

"Reference 'ingested_on' is ambiguous, could be: ingested_on, ingested_on.;"
Traceback (most recent call last):
  File "<stdin>", line 58, in timer_method
  File "/usr/lib64/python3.7/timeit.py", line 232, in timeit
    return Timer(stmt, setup, timer, globals).timeit(number)
  File "/usr/lib64/python3.7/timeit.py", line 176, in timeit
    timing = self.inner(it, self.timer)
  File "<timeit-src>", line 6, in inner
  File "<stdin>", line 9, in get_monthly_totals_pyspark
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1326, in select
    jdf = self._jdf.select(self._jcols(*cols))
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: "Reference 'ingested_on' is ambiguous, co

In [40]:
def get_monthly_totals_pandas(taxiPath, taxiLookupPath):
    taxi = pd.read_json(taxiPath)
    taxi_lookup = pd.read_json(taxiLookupPath)
    taxi_filtered = tax.dropna(subset=['pickup_datetime', 'dropoff_datetime'])
    
    groupDF = taxi_filtered.join(taxi_lookup.set_index('LocationID'), on='PULocationID')
    groupDF['pickup_month'] = pd.to_datetime(groupDF['pickup_datetime'], format='%m%Y')
    groupDF = groupDF.groupby('pickup_month', 'borough').agg('count').sort_values(by=['count', 'borough'], ascending=[False, True])
    groupDF
    return groupDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
print(timer_method("get_monthly_totals_pandas(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

pyarrow and local java libraries required for HDFS
Traceback (most recent call last):
  File "<stdin>", line 58, in timer_method
  File "/usr/lib64/python3.7/timeit.py", line 232, in timeit
    return Timer(stmt, setup, timer, globals).timeit(number)
  File "/usr/lib64/python3.7/timeit.py", line 176, in timeit
    timing = self.inner(it, self.timer)
  File "<timeit-src>", line 6, in inner
  File "<stdin>", line 2, in get_monthly_totals_pandas
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/pandas/util/_decorators.py", line 199, in wrapper
    return func(*args, **kwargs)
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/pandas/util/_decorators.py", line 296, in wrapper
    return func(*args, **kwargs)
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/pandas/io/json/_json.py", line 594, in read_json
    path_or_buf, encoding=encoding, compression=compression
  File "/tmp/1603755040924-0/lib/python3.7/site-packages/pandas/io/common.py", line 222, in get_filepath_or

In [5]:
def get_monthly_totals_pandas(taxiPath, taxiLookupPath):
    taxiPySpark = spark.read.json(taxiPath)
    taxiLookupPySpark = spark.read.json(taxiLookupPath)
    
    taxi = taxiPySpark.toPandas()
    taxiLookup = taxiLookupPySpark.toPandas()
    taxiFiltered = taxi.dropna(subset=['pickup_datetime', 'dropoff_datetime'])
    
    groupDF = taxiFiltered.join(taxiLookup[["Borough", "LocationID"]].set_index('LocationID'), on='PULocationID')
    
    groupDF['pickup_month'] = pd.to_datetime(groupDF['pickup_datetime']).dt.strftime('%Y%m')
    returnGroupDF = groupDF.groupby(['pickup_month', 'Borough']).size().reset_index(name='count').sort_values(by=['pickup_month', 'count', 'Borough'], ascending=[False, False, True])
    return returnGroupDF

def get_monthly_totals_pyspark(taxiPath, taxiLookupPath):
    taxi = spark.read.json(taxiPath)
    taxiLookup = spark.read.json(taxiLookupPath)
    
    taxiFiltered = (taxi
     .filter(taxi.pickup_datetime.isNotNull())
     .filter(taxi.dropoff_datetime.isNotNull()))
                     
    groupDF = taxiFiltered.join(taxiLookup.select("Borough", "LocationID"), taxiFiltered.PULocationID == taxiLookup.LocationID)
    groupDF = groupDF.withColumn("pickup_month", f.date_format("pickup_datetime", "yyyyMM"))
    groupDF = groupDF.groupBy("pickup_month", "borough").count().orderBy(f.desc("pickup_month"), f.desc("count"), "borough")
    groupDF.show()
    return groupDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# Running this command with the original cluster size, will crash the cluster
# All functions utilizing pandas from this command forward, need an upscaled driver node
print(timer_method("get_monthly_totals_pandas(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(527.537092823999,    pickup_month        Borough     count
17       202001      Manhattan  14817800
15       202001       Brooklyn   5628274
18       202001         Queens   4509457
14       202001          Bronx   2536611
20       202001        Unknown   1615247
19       202001  Staten Island    260258
16       202001            EWR      3779
11       201912      Manhattan       129
23       202002      Manhattan        34
5        200901      Manhattan        19
12       201912         Queens        17
6        200901         Queens        10
24       202002         Queens         9
1        200812      Manhattan         8
30       202007      Manhattan         6
26       202003      Manhattan         5
28       202005      Manhattan         5
22       202002       Brooklyn         3
31       202101      Manhattan         3
8        201009         Queens         3
9        201912          Bronx         2
2        200812         Queens         2
4        200901          Bronx        

In [49]:
print(timer_method("get_monthly_totals_pyspark(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+--------+
|pickup_month|      borough|   count|
+------------+-------------+--------+
|      202101|    Manhattan|       3|
|      202007|    Manhattan|       6|
|      202006|    Manhattan|       1|
|      202005|    Manhattan|       5|
|      202004|    Manhattan|       1|
|      202003|    Manhattan|       5|
|      202002|    Manhattan|      34|
|      202002|       Queens|       9|
|      202002|     Brooklyn|       3|
|      202002|        Bronx|       1|
|      202002|      Unknown|       1|
|      202001|    Manhattan|14817800|
|      202001|     Brooklyn| 5628274|
|      202001|       Queens| 4509457|
|      202001|        Bronx| 2536611|
|      202001|      Unknown| 1615247|
|      202001|Staten Island|  260258|
|      202001|          EWR|    3779|
|      201912|    Manhattan|     129|
|      201912|       Queens|      17|
+------------+-------------+--------+
only showing top 20 rows

(33.53619688599974, DataFrame[pickup_month: string, borough: s

In [ ]:
## Expected error for maxResultSize: This won't work. Could try the subsequent cells
## Those restart the state of the notebook and don't work as expected
## Need to restart the cluster and edit the Software config with: [{"classification":"spark-defaults", "properties":{"spark.driver.maxResultSize":"5G", "spark.ui.killEnabled":"true"}, "configurations":[]}]
## Then need to reun the taxi and taxi lookup ingests
## Run -> Run All Above Selected Cell
## Second expected error for {"msg":"requirement failed: Session isn't active."} and will hang. Driver node ran out of mem. Will need to go and upscale
print(spark.conf.get('spark.driver.maxResultSize'))
spark.conf.set("spark.driver.maxResultSize", "5G")
print(spark.conf.get('spark.driver.maxResultSize'))

In [ ]:
%%configure -f
{"conf":{"spark.driver.maxResultSize":"5G"}}

In [ ]:
%%info

In [6]:
def get_monthly_totals_concat_pandas(taxiPath, taxiLookupPath):
    taxiPySpark = spark.read.json(taxiPath)
    taxiLookupPySpark = spark.read.json(taxiLookupPath)
    
    taxi = taxiPySpark.toPandas()
    taxiLookup = taxiLookupPySpark.toPandas()
    taxiFiltered = taxi.dropna(subset=['pickup_datetime', 'dropoff_datetime'])
    
    groupDF = taxiFiltered.join(taxiLookup[["Borough", "LocationID"]].set_index('LocationID'), on='PULocationID')
    groupDF['pickup_month'] = groupDF['year'] + groupDF['month']
    groupDF = groupDF.groupby(['pickup_month', 'Borough']).size().reset_index(name='count').sort_values(by=['pickup_month', 'count', 'Borough'], ascending=[False, False, True])
    return groupDF
    
def get_monthly_totals_concat_pyspark(taxiPath, taxiLookupPath):
    taxi = spark.read.json(taxiPath)
    taxiLookup = spark.read.json(taxiLookupPath)
    taxiFiltered = (taxi
     .filter(taxi.pickup_datetime.isNotNull())
     .filter(taxi.dropoff_datetime.isNotNull()))
        
    groupDF = taxiFiltered.join(taxiLookup, taxiFiltered.PULocationID == taxiLookup.LocationID)
    groupDF = groupDF.withColumn("pickup_month", f.concat("year", "month")).select("pickup_datetime", "borough", "pickup_month")
    groupDF = groupDF.groupBy("pickup_month", "borough").count().orderBy(f.desc("pickup_month"), f.desc("count"), "borough")
    groupDF.show()
    return groupDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
print(timer_method("get_monthly_totals_concat_pandas(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(387.71008044799964,   pickup_month        Borough     count
3       202001      Manhattan  14818011
1       202001       Brooklyn   5628278
4       202001         Queens   4509499
0       202001          Bronx   2536615
6       202001        Unknown   1615251
5       202001  Staten Island    260258
2       202001            EWR      3779)

In [51]:
print(timer_method("get_monthly_totals_concat_pyspark(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+--------+
|pickup_month|      borough|   count|
+------------+-------------+--------+
|      202001|    Manhattan|14818011|
|      202001|     Brooklyn| 5628278|
|      202001|       Queens| 4509499|
|      202001|        Bronx| 2536615|
|      202001|      Unknown| 1615251|
|      202001|Staten Island|  260258|
|      202001|          EWR|    3779|
+------------+-------------+--------+

(17.558869436999885, DataFrame[pickup_month: string, borough: string, count: bigint])

### Lab 2.5 - Case Study 2: Month over month, get the borough with the most amount of pickups per month

In [6]:
def get_most_pickups_per_month_pandas(taxiPath, taxiLookupPath):
    inputDF = get_monthly_totals_pandas(taxiPath, taxiLookupPath)
    firstDF = inputDF.groupby("pickup_month").head()#.first()#sort_values(by=['pickup_month', 'count'], ascending=[True, False]).head(1).reset_index(drop=True)
    firstDF
    return firstDF

def get_most_pickups_per_month_pyspark(taxiPath, taxiLookupPath):
    inputDF = get_monthly_totals_pyspark(taxiPath, taxiLookupPath)
    firstDF = inputDF.orderBy(f.desc("pickup_month"), f.desc("count")).groupBy("pickup_month").agg(f.first("borough")).orderBy(f.desc("pickup_month"))
    firstDF.explain()
    firstDF.show()
    return firstDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
print(timer_method("get_most_pickups_per_month_pandas(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
print(timer_method("get_most_pickups_per_month_pyspark(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+--------+
|pickup_month|      borough|   count|
+------------+-------------+--------+
|      202101|    Manhattan|       3|
|      202007|    Manhattan|       6|
|      202006|    Manhattan|       1|
|      202005|    Manhattan|       5|
|      202004|    Manhattan|       1|
|      202003|    Manhattan|       5|
|      202002|    Manhattan|      34|
|      202002|       Queens|       9|
|      202002|     Brooklyn|       3|
|      202002|        Bronx|       1|
|      202002|      Unknown|       1|
|      202001|    Manhattan|14817800|
|      202001|     Brooklyn| 5628274|
|      202001|       Queens| 4509457|
|      202001|        Bronx| 2536611|
|      202001|      Unknown| 1615247|
|      202001|Staten Island|  260258|
|      202001|          EWR|    3779|
|      201912|    Manhattan|     129|
|      201912|       Queens|      17|
+------------+-------------+--------+
only showing top 20 rows

== Physical Plan ==
AdaptiveSparkPlan(isFinalPlan=false)
+- So

In [31]:
def get_most_pickups_per_month_window_pyspark(taxiPath, taxiLookupPath):
    from pyspark.sql import Window
    inputDF = get_monthly_totals_pyspark(taxiPath, taxiLookupPath)
    win = Window.partitionBy("pickup_month").orderBy(f.desc("count"))
    firstDF = inputDF.withColumn("row_num", f.row_number().over(win)).where("row_num == 1")
    firstDF = firstDF.orderBy(f.desc("pickup_month"))
    firstDF.explain()
    firstDF.show(firstDF.count())
    return firstDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
print(timer_method("get_most_pickups_per_month_window_pyspark(taxiPath, taxiLookupPath)"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+--------+
|pickup_month|      borough|   count|
+------------+-------------+--------+
|      202101|    Manhattan|       3|
|      202007|    Manhattan|       6|
|      202006|    Manhattan|       1|
|      202005|    Manhattan|       5|
|      202004|    Manhattan|       1|
|      202003|    Manhattan|       5|
|      202002|    Manhattan|      34|
|      202002|       Queens|       9|
|      202002|     Brooklyn|       3|
|      202002|        Bronx|       1|
|      202002|      Unknown|       1|
|      202001|    Manhattan|14817800|
|      202001|     Brooklyn| 5628274|
|      202001|       Queens| 4509457|
|      202001|        Bronx| 2536611|
|      202001|      Unknown| 1615247|
|      202001|Staten Island|  260258|
|      202001|          EWR|    3779|
|      201912|    Manhattan|     129|
|      201912|       Queens|      17|
+------------+-------------+--------+
only showing top 20 rows

== Physical Plan ==
AdaptiveSparkPlan(isFinalPlan=false)
+- So

### Lab - 2.6 Run and time the overall pipeline

In [ ]:
# Reset notebook kernel
def ingest_main():
    ingest_taxi_data_multi_service("s3://nyc-tlc/trip data/yellow_tripdata_2020-01.csv")
    ingest_taxi_lookup("s3://nyc-tlc/misc/taxi _zone_lookup.csv")
    get_most_pickups_per_month_window_pyspark()

In [ ]:
print(timer_method("ingest_main"))